In [6]:
MODEL_PATH='/content/drive/My Drive/Tchikou-mer-fay/ML/models/yassir_forecast_model4'

# Install Flask-ngrok

In [2]:
!pip install flask flask-ngrok

# Import libraries

In [16]:
import tensorflow as tf
import pandas as pd
from flask import Flask,render_template,request
from flask_ngrok import run_with_ngrok
import numpy as np
import os,time, datetime
from geopy.distance import vincenty
import os,traceback

# Load the model

In [34]:
# Load the pretrained saved model
# Please change this link according to where your TF model is saved
model = tf.keras.models.load_model(MODEL_PATH)


OSError: ignored

# Start serving

In [8]:
def with_header_response(result,code):
    return result,code,{'Access-Control-Allow-Origin': '*'}


In [30]:
def distanceCalc(row):
    '''calculate distance (km) between two lat&long points using the Vincenty formula '''

    return vincenty((row.pickup_lat, row.pickup_lng), (row.dropoff_lat, row.dropoff_lng)).meters/1000.



def construct_input_df(obj):
    now=datetime.datetime.now()
    date,hour,min=now.day,now.hour,now.minute
    day_time_range=int(hour*4 + ( np.ceil(min/15) ) )
    input_row=pd.DataFrame([obj])
    input_row['distance']=input_row.apply(lambda x: distanceCalc(x), axis=1)
    input_row['day_time_range'] =  input_row.apply(lambda x: day_time_range, axis=1)
    input_row['date'] =  input_row.apply(lambda x: date, axis=1)

    return input_row

def make_prediction(model,pickup_lat,pickup_lng,dropoff_lat,dropoff_lng):
   
    input_raw={
        "pickup_lat":pickup_lat,
        "pickup_lng":pickup_lng,
        "dropoff_lat":dropoff_lat,
        "dropoff_lng":dropoff_lng

    }
    input_row=construct_input_df(input_raw)
    res=float(model.predict(input_row)[0][0])
    print(res,flush=True)
    return {'eta':res }

  


In [31]:

app = Flask(__name__)
run_with_ngrok(app)
@app.route('/estimate',methods=['POST'])
def estimate_ETA():
    params=request.values.to_dict()
    try:
        print(params,flush=True)
        pickup_lat=float(params['depLat'])
        pickup_lng=float(params['depLng'])
        dropoff_lat=float(params['arrLat'])
        dropoff_lng=float(params['arrLng'])
        if 'token' in params.keys():
            result={'eta': float(params['token'])}
    except Exception as e:
        return {'ok': False, 'message': 'Invalid params'},422,{'Access-Control-Allow-Origin': '*'}
    try:
      result=make_prediction(model,pickup_lat,pickup_lng,dropoff_lat,dropoff_lng)
      print(result,flush=True)
    except Exception as e: 
      print('Error occured when predicting : ',flush=True)
      traceback.print_exc()
      
      return with_header_response({'ok' : False, 'message': 'An error occured'},500)    

    
    return result,200,{'Access-Control-Allow-Origin': '*'}


 # For Access origin issues
@app.after_request
def after_request(response):
    header = response.headers
    header['Access-Control-Allow-Origin'] = '*'
    # header['Access-Control-Allow-Headers'] = 'Content-Type, Authorization'
    header['Access-Control-Allow-Methods'] = 'OPTIONS, HEAD, GET, POST, DELETE, PUT'
    response.headers=header
    return response

app.run()    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d79ed5125d92.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{'depLat': '36.763215', 'depLng': '3.042545', 'arrLat': '36.766107', 'arrLng': '3.004864', 'token': '23.99'}
15996.216796875
{'eta': 15996.216796875}


127.0.0.1 - - [25/Dec/2020 23:06:54] "POST /estimate HTTP/1.1" 200 -


{'depLat': '36.767454', 'depLng': '3.035457', 'arrLat': '36.767877', 'arrLng': '3.004798', 'token': '26.73'}
16138.4169921875
{'eta': 16138.4169921875}


127.0.0.1 - - [25/Dec/2020 23:22:41] "POST /estimate HTTP/1.1" 200 -
